In [200]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, Normalizer

from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_percentage_error
from pmdarima import auto_arima
import random
import itertools
import statsmodels.api as sm
from pandas.plotting import register_matplotlib_converters
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error,mean_absolute_error
import warnings
import lightgbm as LGBMRegressor
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error

from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.decomposition import PCA, KernelPCA
from tqdm import tqdm
import itertools

In [201]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
building = pd.read_csv('./building_info.csv')

In [202]:
train = train.rename(columns={
    '건물번호': 'building_number',
    '일시': 'date',
    '기온(C)': 'temperature',
    '강수량(mm)': 'rainfall',
    '풍속(m/s)': 'windspeed',
    '습도(%)': 'humidity',
    '일조(hr)': 'sunshine',
    '일사(MJ/m2)': 'solar_radiation',
    '전력소비량(kWh)': 'power_consumption'
})
train.drop('num_date_time', axis = 1, inplace=True)

test = test.rename(columns={
    '건물번호': 'building_number',
    '일시': 'date',
    '기온(C)': 'temperature',
    '강수량(mm)': 'rainfall',
    '풍속(m/s)': 'windspeed',
    '습도(%)': 'humidity',
    '일조(hr)': 'sunshine',
    '일사(MJ/m2)': 'solar_radiation',
    '전력소비량(kWh)': 'power_consumption'
})
test.drop('num_date_time', axis = 1, inplace=True)

building = building.rename(columns={
    '건물번호': 'building_number',
    '건물유형': 'building_type',
    '연면적(m2)': 'total_area',
    '냉방면적(m2)': 'cooling_area',
    '태양광용량(kW)': 'solar_power_capacity',
    'ESS저장용량(kWh)': 'ess_capacity',
    'PCS용량(kW)': 'pcs_capacity'
})


In [203]:
train = pd.merge(train, building, on='building_number', how='left')
test = pd.merge(test, building, on='building_number', how='left')

In [204]:
train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])

In [205]:
train['date'].unique()

array(['2022-06-01T00:00:00.000000000', '2022-06-01T01:00:00.000000000',
       '2022-06-01T02:00:00.000000000', ...,
       '2022-08-24T21:00:00.000000000', '2022-08-24T22:00:00.000000000',
       '2022-08-24T23:00:00.000000000'], dtype='datetime64[ns]')

In [206]:
train['rainfall'] = train['rainfall'].fillna(0)
train['sunshine'] = train['sunshine'].fillna(0)
train['solar_radiation'] = train['solar_radiation'].fillna(0)

In [207]:
weekend = [0 for _ in range(len(train))]

for i in train.query('date.dt.dayofweek == 6').index:
    weekend[i] = 1
    
for i in train.query('date.dt.dayofweek == 5').index:
    weekend[i] = 1
    
train['sudo_weekend'] = weekend
train

,building_number,date,temperature,rainfall,windspeed,humidity,sunshine,solar_radiation,power_consumption,building_type,total_area,cooling_area,solar_power_capacity,ess_capacity,pcs_capacity,sudo_weekend
0,1,2022-06-01 00:00:00,18.6,0.0,0.9,42.0,0.0,0.0,1085.28,건물기타,110634.00,39570.00,-,-,-,0
1,1,2022-06-01 01:00:00,18.0,0.0,1.1,45.0,0.0,0.0,1047.36,건물기타,110634.00,39570.00,-,-,-,0
2,1,2022-06-01 02:00:00,17.7,0.0,1.5,45.0,0.0,0.0,974.88,건물기타,110634.00,39570.00,-,-,-,0
3,1,2022-06-01 03:00:00,16.7,0.0,1.4,48.0,0.0,0.0,953.76,건물기타,110634.00,39570.00,-,-,-,0
4,1,2022-06-01 04:00:00,18.4,0.0,2.8,43.0,0.0,0.0,986.40,건물기타,110634.00,39570.00,-,-,-,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100,2022-08-24 19:00:00,23.1,0.0,0.9,86.0,0.5,0.0,881.04,호텔및리조트,57497.84,40035.23,-,-,-,0
203996,100,2022-08-24 20:00:00,22.4,0.0,1.3,86.0,0.0,0.0,798.96,호텔및리조트,57497.84,40035.23,-,-,-,0
203997,100,2022-08-24 21:00:00,21.3,0.0,1.0,92.0,0.0,0.0,825.12,호텔및리조트,57497.84,40035.23,-,-,-,0
203998,100,2022-08-24 22:00:00,21.0,0.0,0.3,94.0,0.0,0.0,640.08,호텔및리조트,57497.84,40035.23,-,-,-,0


In [208]:
# 6월6일 8월15일 공휴일

holiday = [f'2022-06-06 0{i}:00:00' for i in range(10)]
holiday.extend([f'2022-06-06 {i}:00:00' for i in range(10,24)])
holiday.extend([f'2022-08-15 0{i}:00:00' for i in range(10)])
holiday.extend([f'2022-08-15 {i}:00:00' for i in range(10,24)])
holiday

['2022-06-06 00:00:00',
 '2022-06-06 01:00:00',
 '2022-06-06 02:00:00',
 '2022-06-06 03:00:00',
 '2022-06-06 04:00:00',
 '2022-06-06 05:00:00',
 '2022-06-06 06:00:00',
 '2022-06-06 07:00:00',
 '2022-06-06 08:00:00',
 '2022-06-06 09:00:00',
 '2022-06-06 10:00:00',
 '2022-06-06 11:00:00',
 '2022-06-06 12:00:00',
 '2022-06-06 13:00:00',
 '2022-06-06 14:00:00',
 '2022-06-06 15:00:00',
 '2022-06-06 16:00:00',
 '2022-06-06 17:00:00',
 '2022-06-06 18:00:00',
 '2022-06-06 19:00:00',
 '2022-06-06 20:00:00',
 '2022-06-06 21:00:00',
 '2022-06-06 22:00:00',
 '2022-06-06 23:00:00',
 '2022-08-15 00:00:00',
 '2022-08-15 01:00:00',
 '2022-08-15 02:00:00',
 '2022-08-15 03:00:00',
 '2022-08-15 04:00:00',
 '2022-08-15 05:00:00',
 '2022-08-15 06:00:00',
 '2022-08-15 07:00:00',
 '2022-08-15 08:00:00',
 '2022-08-15 09:00:00',
 '2022-08-15 10:00:00',
 '2022-08-15 11:00:00',
 '2022-08-15 12:00:00',
 '2022-08-15 13:00:00',
 '2022-08-15 14:00:00',
 '2022-08-15 15:00:00',
 '2022-08-15 16:00:00',
 '2022-08-15 17:

In [209]:
holy_idx = [train[train['date']==i].index for i in holiday]

ans = [0 for _ in range(204000)]
for t in holy_idx:
    for idx in t:
        ans[idx] = 1

train['sudo_holiday'] = ans
train

,building_number,date,temperature,rainfall,windspeed,humidity,sunshine,solar_radiation,power_consumption,building_type,total_area,cooling_area,solar_power_capacity,ess_capacity,pcs_capacity,sudo_weekend,sudo_holiday
0,1,2022-06-01 00:00:00,18.6,0.0,0.9,42.0,0.0,0.0,1085.28,건물기타,110634.00,39570.00,-,-,-,0,0
1,1,2022-06-01 01:00:00,18.0,0.0,1.1,45.0,0.0,0.0,1047.36,건물기타,110634.00,39570.00,-,-,-,0,0
2,1,2022-06-01 02:00:00,17.7,0.0,1.5,45.0,0.0,0.0,974.88,건물기타,110634.00,39570.00,-,-,-,0,0
3,1,2022-06-01 03:00:00,16.7,0.0,1.4,48.0,0.0,0.0,953.76,건물기타,110634.00,39570.00,-,-,-,0,0
4,1,2022-06-01 04:00:00,18.4,0.0,2.8,43.0,0.0,0.0,986.40,건물기타,110634.00,39570.00,-,-,-,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100,2022-08-24 19:00:00,23.1,0.0,0.9,86.0,0.5,0.0,881.04,호텔및리조트,57497.84,40035.23,-,-,-,0,0
203996,100,2022-08-24 20:00:00,22.4,0.0,1.3,86.0,0.0,0.0,798.96,호텔및리조트,57497.84,40035.23,-,-,-,0,0
203997,100,2022-08-24 21:00:00,21.3,0.0,1.0,92.0,0.0,0.0,825.12,호텔및리조트,57497.84,40035.23,-,-,-,0,0
203998,100,2022-08-24 22:00:00,21.0,0.0,0.3,94.0,0.0,0.0,640.08,호텔및리조트,57497.84,40035.23,-,-,-,0,0


In [210]:
# pd.date_range('2022-06-01 00:00:00','2022-08-24 23:00:00',freq='h')

In [211]:
# data = [train[train['building_type'] == i].reset_index(drop=True) for i in train.building_type.unique()]


In [212]:
train

,building_number,date,temperature,rainfall,windspeed,humidity,sunshine,solar_radiation,power_consumption,building_type,total_area,cooling_area,solar_power_capacity,ess_capacity,pcs_capacity,sudo_weekend,sudo_holiday
0,1,2022-06-01 00:00:00,18.6,0.0,0.9,42.0,0.0,0.0,1085.28,건물기타,110634.00,39570.00,-,-,-,0,0
1,1,2022-06-01 01:00:00,18.0,0.0,1.1,45.0,0.0,0.0,1047.36,건물기타,110634.00,39570.00,-,-,-,0,0
2,1,2022-06-01 02:00:00,17.7,0.0,1.5,45.0,0.0,0.0,974.88,건물기타,110634.00,39570.00,-,-,-,0,0
3,1,2022-06-01 03:00:00,16.7,0.0,1.4,48.0,0.0,0.0,953.76,건물기타,110634.00,39570.00,-,-,-,0,0
4,1,2022-06-01 04:00:00,18.4,0.0,2.8,43.0,0.0,0.0,986.40,건물기타,110634.00,39570.00,-,-,-,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100,2022-08-24 19:00:00,23.1,0.0,0.9,86.0,0.5,0.0,881.04,호텔및리조트,57497.84,40035.23,-,-,-,0,0
203996,100,2022-08-24 20:00:00,22.4,0.0,1.3,86.0,0.0,0.0,798.96,호텔및리조트,57497.84,40035.23,-,-,-,0,0
203997,100,2022-08-24 21:00:00,21.3,0.0,1.0,92.0,0.0,0.0,825.12,호텔및리조트,57497.84,40035.23,-,-,-,0,0
203998,100,2022-08-24 22:00:00,21.0,0.0,0.3,94.0,0.0,0.0,640.08,호텔및리조트,57497.84,40035.23,-,-,-,0,0


In [215]:
building[50:50*2]

,building_number,building_type,total_area,cooling_area,solar_power_capacity,ess_capacity,pcs_capacity
50,51,병원,88242.760,80566.00,-,-,-
51,52,병원,91634.210,69227.19,25,-,-
52,53,상용,149246.000,110759.00,-,-,-
53,54,상용,109400.200,65803.57,-,-,-
54,55,상용,25341.870,13040.10,-,-,-
55,56,상용,42666.540,7259.10,-,-,-
56,57,상용,105473.460,55586.72,200,-,-
57,58,상용,14897.000,7071.92,9.45,-,-
58,59,상용,105500.720,84402.00,-,-,-
59,60,상용,188902.070,142187.28,365,-,-
